## Q1

In [1]:
import numpy as np

In [2]:
input = np.array([[2,0,2,1,3,2],
                  [0,2,0,2,2,2],
                  [1,0,1,3,1,1],
                  [0,0,1,1,1,0],
                  [0,1,3,4,1,0],
                  [0,1,0,0,5,2]])

filter = np.array([[1,0,0],
                   [0,1,0],
                   [0,0,1]])

In [4]:
padding = np.pad(input, ((1,1),(1,1)), 'constant', constant_values=0) #위, 아래, 왼쪽, 오른쪽

In [21]:
padding

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
filter

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

## filter

In [61]:
result=np.zeros((6,6))

for i in range(6):
    for j in range(6):
        result[i,j]=(np.sum(padding[i:i+3, j:j+3] * filter))
result

array([[ 4.,  0.,  4.,  3.,  5.,  2.],
       [ 0.,  5.,  3.,  5.,  4.,  5.],
       [ 1.,  1.,  4.,  4.,  3.,  3.],
       [ 1.,  4.,  5.,  3.,  4.,  1.],
       [ 1.,  1.,  3., 10.,  4.,  1.],
       [ 0.,  1.,  1.,  3.,  9.,  3.]])

## maxpooling

In [62]:
result2=[]
for i in range(0,5,2):
    for j in range(0,5,2):
        result2.append(np.amax(np.array(result[i:i+2, j:j+2])))

In [64]:
np.array(result2).reshape(3,3)

array([[ 5.,  5.,  5.],
       [ 4.,  5.,  4.],
       [ 1., 10.,  9.]])

## Q2

In [65]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [66]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [67]:
# parameter setting

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [68]:
# dataset load

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = False)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = False)

In [69]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [70]:
#CNN Model (2 conv layers)
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    #flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [71]:
#instantiate CNN Model
model = CNN().to(device)

In [72]:
criterion = torch.nn.CrossEntropyLoss().to(device)    #Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [73]:
#model training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        #image is already size of (28x28), no reshape
        #label is not one-hot coded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.190713644
[Epoch:    2] cost = 0.0520608276
[Epoch:    3] cost = 0.0376487821
[Epoch:    4] cost = 0.0285026077
[Epoch:    5] cost = 0.0243023224
[Epoch:    6] cost = 0.0195404254
[Epoch:    7] cost = 0.0160752013
[Epoch:    8] cost = 0.0142789194
[Epoch:    9] cost = 0.0117072025
[Epoch:   10] cost = 0.0123375095
[Epoch:   11] cost = 0.0106206443
[Epoch:   12] cost = 0.0107046869
[Epoch:   13] cost = 0.00735590933
[Epoch:   14] cost = 0.00858276151
[Epoch:   15] cost = 0.00802615285
Learning Finished!


In [74]:
#accuracy check
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9829999804496765


## layer1: kernel size=4 , layer3: stride=2

In [80]:
#CNN Model
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(2 * 2 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    #flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [81]:
#instantiate CNN Model
model = CNN().to(device)

In [82]:
criterion = torch.nn.CrossEntropyLoss().to(device)    #Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [83]:
#model training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        #image is already size of (28x28), no reshape
        #label is not one-hot coded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.242068917
[Epoch:    2] cost = 0.0637366921
[Epoch:    3] cost = 0.0441345572
[Epoch:    4] cost = 0.0347181968
[Epoch:    5] cost = 0.0287644975
[Epoch:    6] cost = 0.0224166028
[Epoch:    7] cost = 0.0201460645
[Epoch:    8] cost = 0.0174987428
[Epoch:    9] cost = 0.0145963728
[Epoch:   10] cost = 0.0151204374
[Epoch:   11] cost = 0.0111700008
[Epoch:   12] cost = 0.0111832069
[Epoch:   13] cost = 0.011417822
[Epoch:   14] cost = 0.010588143
[Epoch:   15] cost = 0.00789844524
Learning Finished!


## layer3 size

(2,2,128)

## 성능비교

In [84]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9861000180244446


In [85]:
#약간 상승